In [199]:
import json
import requests
import csv
import time
import pprint
import pickle

In [200]:
# API_KEY = "bee552bc-c708-4375-9e34-01662a8822e0"
def getinfo(call):
    r = requests.get(call,timeout=10)
    return r.json()
with open('Snipers.json') as f:
    snipers = json.load(f)
with open('Players.json') as f:
    players = json.load(f)

In [201]:
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, item in enumerate(v):
                items.extend(flatten_dict({str(i): item}, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [202]:
def get_data(uuids, Snipe):
    # uuids = ["1ced16db-2bc5-4354-917f-7bf8382b8840"]
    data_dic_list = []
    with open('status.json') as f:
        di = json.load(f)
    with open('api.json') as f:
        tokens = json.load(f)

    for uuid in uuids:
        ut = time.time()
        if ut - di["callstart"] >= 300:
            di["callstart"] = ut
            di["call"] = 0
        di["call"] += 1
        with open('status.json', 'w') as f:
            json.dump(di, f)
        API_KEY = tokens[f"HYPIXEL_TOKEN_{di['api_num']}"]
        if di["call"] >= 290:
            if di["api_num"] == 0:
                API_KEY = tokens["HYPIXEL_TOKEN_1"]
                di["api_num"] = 1
            elif di["api_num"] == 1:
                API_KEY = tokens["HYPIXEL_TOKEN_2"]
                di["api_num"] = 2
            elif di["api_num"] == 2:
                API_KEY = tokens["HYPIXEL_TOKEN_0"]
                di["api_num"] = 0
            di["call"] = 0
            with open('status.json', 'w') as f:
                json.dump(di, f)

        uuid_link = f"https://api.hypixel.net/player?key={API_KEY}&uuid={uuid}"
        data_dic = getinfo(uuid_link)
        # pprint.pprint(data_dic)
        if data_dic is not None and "cause" in data_dic and data_dic["cause"] == "Key throttle":
            time.sleep(60)
            continue
        if data_dic is not None and data_dic["success"] == True:
            data_dic["Snipe"] = Snipe
            flattened_data = flatten_dict(data_dic)
            data_dic_list.append(flattened_data)
            # pprint.pprint(data_dic)
    return data_dic_list

In [203]:
data_dic_list = get_data(snipers, True)
data_dic_list += get_data(players, False)

In [204]:
# with open('Player_Data.json', 'w') as f:
#     json.dump(data_dic_list, f)

In [205]:
flattened_data_list = []
for dic in data_dic_list:
    # 辞書をフラット化する
    flattened_data = flatten_dict(dic)
    flattened_data_list.append(flattened_data)

In [211]:
labels_dic = {}
for dic in flattened_data_list:
    for label in dic:
        if not label in labels_dic:
            labels_dic[label] = type(dic[label])
with open('models/labels_dic.pkl', 'wb') as f:
    pickle.dump(labels_dic, f)

In [207]:
for index, dic in enumerate(flattened_data_list):
    for label in labels_dic:
        if not label in dic:
            if labels_dic[label] == str:
                flattened_data_list[index][label] = "Null"
                # print("str")
            elif labels_dic[label] == int:
                flattened_data_list[index][label] = 0
                # print("int")
            elif labels_dic[label] == float:
                flattened_data_list[index][label] = 0.0
                # print("float")
            elif labels_dic[label] == bool:
                flattened_data_list[index][label] = False
                # print("bool")
            elif labels_dic[label] == None:
                flattened_data_list[index][label] = None
                # print("None")
# pprint.pprint(flattened_data_list)

In [208]:
# with open('Sniper_Data.json', 'w') as f:
#     json.dump(flattened_data_list, f)

In [209]:
# with open('Sniper_Data.json') as f:
#     di = json.load(f)
#     pprint.pprint(di[1])

In [210]:
# CSVファイル名
csv_file = "Data.csv"

# 辞書のラベル
fieldnames = labels_dic.keys()

# CSVファイルに書き込み
with open(csv_file, mode="w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for d in flattened_data_list:
        writer.writerow(d)